In [164]:
import numpy as np
import pandas as pd
import re
import csv
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import regularizers, initializers, optimizers, callbacks
import nltk
nltk.download('punkt')
import re
import string
punctuation = string.punctuation
from nltk.corpus import stopwords
stopwords = stopwords.words("english")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aviralsharma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [165]:
celeb = pd.read_csv("celeb.csv")

In [166]:
def cleanTweet(tweet):
    tweet = tweet.lower()
    #maybe we want to remove the twitter handle as well
    #removing the punctuations
    tweet = "".join(x for x in tweet if x not in punctuation)
    #removing the stop words
    individualWords = tweet.split()
    individualWords = [w for w in individualWords if w not in stopwords]
    tweet = " ".join(individualWords)
    #removing certain common occuring words
    tweet = re.sub('rt', '', tweet)
    return tweet

In [167]:
celeb = celeb[celeb["language"] == 'en']
celeb = celeb[["author", "content"]]
celeb["clean tweets"] = celeb["content"].apply(cleanTweet)
celeb = celeb.sample(frac=1).reset_index(drop=True)
celeb = celeb[:6000]
celeb.to_csv("CelebrityTweets.csv")

In [10]:
from keras.models import model_from_json
# Load the model
emotion_file = open('twitter.json', 'r')
loaded_model_json = emotion_file.read()
emotion_file.close()
modeled_emotion = model_from_json(loaded_model_json)

# Load the weights
modeled_emotion.load_weights('twitter.h5')

W1129 13:09:11.306292 4771452352 deprecation_wrapper.py:119] From /Users/aviralsharma/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1129 13:09:11.336508 4771452352 deprecation_wrapper.py:119] From /Users/aviralsharma/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1129 13:09:11.438102 4771452352 deprecation_wrapper.py:119] From /Users/aviralsharma/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1129 13:09:11.595080 4771452352 deprecation_wrapper.py:119] From /Users/aviralsharma/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session inst

In [168]:
tweets = celeb["clean tweets"]
t = []
for i in tweets:
    t.append(i)
t

['know love pickles ship home 🙈 bestmaidpickles texastradition texasgirl httptcolkornkcowk',
 'barmy117 smurftastic lol thats cute oohlalavideopremiere',
 'asked ecuador loves httptcojqdbnrdxug',
 'touring w nickjonas best getting perform song w even better httpstco5bszcnehmz hondacivictour futurenow',
 'foo fighters incredible tonight monster fury rock n roll 20 years locked precision… httptcovf97dfo3cy',
 '🐠🐟had magical dinner tour watching natural choreography beautiful sea life in… httpstcojyx1zx9lfx',
 'love httpstcopmulwsr1cs',
 'tonight jon stewa padmalakshmi plus standup jimgaffigan fallontonight',
 '“gwenmarie4936 honestly flipping excited album come 😝❤ preordered waiting httptcongbuhdyfax”',
 'tomorrow sidetoside ♡🚲 httpstcou6xjvscikc',
 'haaaa comment thank “sweetyitosan katyperry omg prism purrfect like eaten angel prism',
 'happy bihday debut albums fame amp fame monster🍾🍰 httpstcolstif0fkbz',
 'economy added 14 million jobs longest streak privatesector job growth record h

In [17]:
MAX_NB_WORDS = 40000
MAX_SEQUENCE_LENGTH = 30
VALIDATION_SPLIT = 0.2

In [141]:

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(t)
sequences = tokenizer.texts_to_sequences(t)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


In [169]:
probs = []
# for x in data:
#     print(x.shape)
z = 1
r = 0
while z != len(t) + 1:
        y = modeled_emotion.predict(data[r:z])
        probs.append(y)
        r += 1
        z += 1

In [170]:
len(probs)

6000

In [147]:
len(t)

6000

In [171]:
celeb["probs"] = probs

In [154]:
# celeb['probs'][0][0] for accessing the probabilities

0.03297202

In [175]:
celeb.to_csv("FinalCelebrityTweets.csv")

,author,content,clean tweets,probs
0,selenagomez,You know you love pickles when you ship them f...,know love pickles ship home 🙈 bestmaidpickles ...,"[[0.03297202, 0.041923206, 0.094275996, 0.6300..."
1,britneyspears,"@BARMY117 Smurftastic, lol. That's cute ;) #Oo...",barmy117 smurftastic lol thats cute oohlalavid...,"[[0.28463885, 0.301798, 0.12617503, 0.02485458..."
2,Cristiano,You asked: Ecuador loves you!!! http://t.co/j...,asked ecuador loves httptcojqdbnrdxug,"[[0.0049851704, 0.0065451604, 0.021164875, 0.3..."
3,ddlovato,Touring w/ @nickjonas was the BEST. Getting to...,touring w nickjonas best getting perform song ...,"[[0.009394343, 0.01196938, 0.03215504, 0.40713..."
4,ladygaga,"The Foo Fighters were so incredible tonight, m...",foo fighters incredible tonight monster fury r...,"[[0.018530058, 0.022273604, 0.031897236, 0.165..."
5,katyperry,🐠🐟Had a magical dinner with the tour watching ...,🐠🐟had magical dinner tour watching natural cho...,"[[0.00023873808, 0.00033093386, 0.0011966779, ..."
6,ArianaGrande,and i love you more https://t.co/pMUlWsR1cS,love httpstcopmulwsr1cs,"[[0.124220766, 0.13731481, 0.12941815, 0.30557..."
7,jimmyfallon,Tonight: Jon Stewart and @PadmaLakshmi are her...,tonight jon stewa padmalakshmi plus standup ji...,"[[0.007611563, 0.008999298, 0.008821664, 0.025..."
8,jtimberlake,“@GwenMarie_4936: I am honestly so flipping ex...,“gwenmarie4936 honestly flipping excited album...,"[[0.30922565, 0.3172511, 0.12342917, 0.0097290..."
9,ArianaGrande,tomorrow #SideToSide ♡🚲 https://t.co/U6XjVSCIKc,tomorrow sidetoside ♡🚲 httpstcou6xjvscikc,"[[0.13839291, 0.15513825, 0.1445777, 0.2630746..."
